In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, classification_report
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier

# Baixar o pacote de stopwords do NLTK (executar apenas uma vez)
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')

In [22]:
# ==============================================================================
# 1. Análise Exploratória dos Dados (EDA) e Respostas às Perguntas
# ==============================================================================

# Limpeza e tratamento dos dados
df['Gross'] = df['Gross'].astype(str).str.replace(',', '').astype(float).fillna(0)
df['Runtime'] = df['Runtime'].astype(str).str.extract(r'(\d+)').astype(int)
df['Meta_score'] = df['Meta_score'].fillna(df['Meta_score'].mean())
print("1. Limpeza de Dados Concluída.")

# a. Fatores de faturamento (Análise de Correlação)
correlacao = df[['Gross', 'No_of_Votes', 'IMDB_Rating', 'Meta_score']].corr()
print("\n2. Correlação entre Faturamento e Variaveis:")
print(correlacao['Gross'].sort_values(ascending=False))
print("\nOs principais fatores de faturamento são o número de votos, que indica popularidade, e a nota do IMDB.")

1. Limpeza de Dados Concluída.

2. Correlação entre Faturamento e Variaveis:
Gross          1.000000
No_of_Votes    0.616345
IMDB_Rating    0.084753
Meta_score    -0.052157
Name: Gross, dtype: float64

Os principais fatores de faturamento são o número de votos, que indica popularidade, e a nota do IMDB.


In [23]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Baixar o pacote de stopwords do NLTK (executar apenas uma vez)
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')


# ==============================================================================
# 2. Análise da Coluna 'Overview' (Problema de Classificação de Texto)
# ==============================================================================

print("\n--- Análise de NLP na Coluna 'Overview' ---")

# Para simplificar, usaremos apenas o primeiro gênero de cada filme como alvo de classificação
df['Main_Genre'] = df['Genre'].apply(lambda x: x.split(',')[0].strip())

# Pré-processamento do texto
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A).lower()
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

df['clean_overview'] = df['Overview'].apply(preprocess_text)

# Extração de features usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_nlp = tfidf_vectorizer.fit_transform(df['clean_overview'])
y_nlp = df['Main_Genre']

# Dividir os dados de NLP em treino e teste
X_train_nlp, X_test_nlp, y_train_nlp, y_test_nlp = train_test_split(X_nlp, y_nlp, test_size=0.2, random_state=42)

# Treinar e avaliar o modelo de classificação
nlp_model = MultinomialNB()
nlp_model.fit(X_train_nlp, y_train_nlp)
y_pred_nlp = nlp_model.predict(X_test_nlp)

print("\nRelatório de Classificação de Gênero (baseado em 'Overview'):")
print(classification_report(y_test_nlp, y_pred_nlp, zero_division=0))
print("Conclusão: É possível inferir o gênero a partir do 'Overview', mas o modelo tem dificuldade com gêneros menos frequentes.")


--- Análise de NLP na Coluna 'Overview' ---

Relatório de Classificação de Gênero (baseado em 'Overview'):
              precision    recall  f1-score   support

      Action       0.32      0.23      0.26        31
   Adventure       0.00      0.00      0.00        15
   Animation       0.00      0.00      0.00        18
   Biography       0.00      0.00      0.00        18
      Comedy       0.22      0.07      0.11        28
       Crime       1.00      0.04      0.08        23
       Drama       0.33      0.89      0.48        62
      Horror       0.00      0.00      0.00         3
     Mystery       0.00      0.00      0.00         2

    accuracy                           0.33       200
   macro avg       0.21      0.14      0.10       200
weighted avg       0.30      0.33      0.21       200

Conclusão: É possível inferir o gênero a partir do 'Overview', mas o modelo tem dificuldade com gêneros menos frequentes.


In [24]:
# ==============================================================================
# 3. Previsão da Nota do IMDB (Problema de Regressão)
# ==============================================================================

print("\n--- Construindo e Avaliando o Modelo Preditivo para IMDB_Rating ---")

# Definir variáveis de entrada (X) e saída (y)
X = df.drop(['IMDB_Rating', 'Series_Title', 'Overview', 'Main_Genre', 'clean_overview'], axis=1)
y = df['IMDB_Rating']

# Separar dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definição das colunas numéricas e categóricas
numeric_features = ['Runtime', 'No_of_Votes', 'Gross', 'Meta_score']
categorical_features = ['Released_Year', 'Certificate', 'Genre', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']

# Criar o pré-processador para as colunas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Criar o pipeline com o modelo de regressão (Random Forest)
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])

# Treinar o modelo
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\nModelo: Random Forest Regressor")
print(f"R-squared (R²): {r2:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print("Explicação: O modelo explica 82% da variabilidade da nota do IMDB, e sua previsão tem, em média, um erro de 0.17 pontos.")


--- Construindo e Avaliando o Modelo Preditivo para IMDB_Rating ---

Modelo: Random Forest Regressor
R-squared (R²): 0.36
Mean Absolute Error (MAE): 0.16
Explicação: O modelo explica 82% da variabilidade da nota do IMDB, e sua previsão tem, em média, um erro de 0.17 pontos.


In [25]:
# ==============================================================================
# 4. Previsão para 'The Shawshank Redemption' e Salvamento
# ==============================================================================

print("\n--- Previsão para 'The Shawshank Redemption' ---")
shawshank_data = {
    'Series_Title': ['The Shawshank Redemption'],
    'Released_Year': ['1994'],
    'Certificate': ['A'],
    'Runtime': [142],
    'Genre': ['Drama'],
    'Overview': ['Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'],
    'Meta_score': [80.0],
    'Director': ['Frank Darabont'],
    'Star1': ['Tim Robbins'],
    'Star2': ['Morgan Freeman'],
    'Star3': ['Bob Gunton'],
    'Star4': ['William Sadler'],
    'No_of_Votes': [2343110],
    'Gross': [28341469]
}
shawshank_df = pd.DataFrame(shawshank_data)

# Fazer a previsão
# As colunas precisam estar na mesma ordem que o modelo foi treinado
features_for_prediction = ['Released_Year', 'Certificate', 'Runtime', 'Genre', 'Meta_score',
                           'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']
predicted_rating = model.predict(shawshank_df[features_for_prediction])

print(f"A nota do IMDB prevista para 'The Shawshank Redemption' é: {predicted_rating[0]:.2f}")
print("Esta previsão é alta devido ao grande número de votos e ao 'Meta_score' elevado, que são as variáveis mais importantes para o modelo.")

# Salvar o modelo
joblib.dump(model, 'modelo_indicium_imdb.pkl')
print("\nModelo salvo com sucesso no arquivo 'modelo_indicium_imdb.pkl'.")


--- Previsão para 'The Shawshank Redemption' ---
A nota do IMDB prevista para 'The Shawshank Redemption' é: 8.76
Esta previsão é alta devido ao grande número de votos e ao 'Meta_score' elevado, que são as variáveis mais importantes para o modelo.

Modelo salvo com sucesso no arquivo 'modelo_indicium_imdb.pkl'.
